In [1]:
import os
import string
import cv2
import fnmatch
import numpy as np
import h5py

In [2]:
char_list = string.ascii_letters + string.digits
print(char_list)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789


In [3]:
def encode_to_labels(text):
    code = []
    for index, char in enumerate(text):
        code.append(char_list.index(char))
    return code

In [4]:
path = 'E:/OCR/CRNN_MJSynth/mjsynth/mnt/ramdisk/max/90kDICT32px'
#path = '/media/monica/New Volume/OCR/CRNN_MJSynth/mjsynth/mnt/ramdisk/max/90kDICT32px'

#for training dataset
train_img_addr = []
train_labels = []

#for validation dataset
test_img_addr = []
test_labels = []

max_label_len = 0

i = 1
flag = 0

for root, dirnames, filenames in os.walk(path):
    
    for f_name in fnmatch.filter(filenames, '*.jpg'):
        label = f_name.split('_')[1]
         # split the 150000 data into validation and training dataset as 10% and 90% respectively
        if i%10 == 0:     
            test_img_addr.append(os.path.join(root, f_name))
            test_labels.append(label)
        else:
            train_img_addr.append(os.path.join(root, f_name))
            train_labels.append(label) 
        
        # break the loop if total data is 150000
        if i == 150000:
            flag = 1
            break
        i+=1
    if flag == 1:
        break

print(len(test_img_addr))
print(len(test_labels))
print(len(train_img_addr))
print(len(train_labels))

15000
15000
135000
135000


In [5]:
train_labels[:8]

['Classmates',
 'greater',
 'BULLFROG',
 'NUCLEAR',
 'maximilian',
 'reedy',
 'reformulated',
 'Insulators']

In [6]:
train_shape = (len(train_img_addr), 32, 128, 1)
test_shape = (len(test_img_addr), 32, 128, 1)

hdf5_path = 'E:/OCR/CRNN_MJSynth/mjsynth_150000.hdf5'
#hdf5_path = "/media/monica/New Volume/OCR/mjsynth_1500.hdf5"
f = h5py.File(hdf5_path, mode='w')

In [7]:
f.create_dataset("train_img", train_shape)
f.create_dataset("test_img", test_shape)

str_type = h5py.new_vlen(str) 
f.create_dataset("train_labels", (len(train_img_addr), ), dtype=str_type)
f['train_labels'][...] = train_labels

f.create_dataset("test_labels", (len(test_img_addr), ), dtype=str_type)
f['test_labels'][...] = test_labels

C:\Anaconda3\envs\deep-learning\lib\site-packages\ipykernel_launcher.py:4: H5pyDeprecationWarning: Deprecated; use special_dtype(vlen=basetype) instead
  after removing the cwd from sys.path.


In [8]:
for key in f.keys():
        print(key)

test_img
test_labels
train_img
train_labels


In [9]:
j = 0
for i in range(len(train_img_addr)):

    addr = train_img_addr[i]
    img = cv2.imread(addr)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
            # reshape image to (32,128,1)
    #print(type(img),img.shape, train_labels[i])
    w, h = img.shape
    flag = False
    
    if h > 128 or w > 32:
        continue
    j += 1
    if w < 32:
        add_zeros = np.ones((32-w, h))*255
        img = np.concatenate((img, add_zeros))
        flag = True
        #img = np.expand_dims(img , axis = 2)
 
    if h < 128:
        add_zeros = np.ones((32, 128-h))*255
        img = np.concatenate((img, add_zeros), axis=1)
        img = np.expand_dims(img , axis = 2)
        flag = True
    if h == 128 or w == 32 and not flag:
        img = np.expand_dims(img , axis = 2)
            # Normalize each image
    img = img/255.
    #print(img.shape, train_labels[i])
    f["train_img"][i, ...] = img[None] 
print ('Train data: {}/{}'.format(j, len(train_img_addr)) )

# loop over test paths
j = 0
for i in range(len(test_img_addr)):

    addr = test_img_addr[i]
    img = cv2.imread(addr)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
            # reshape image to (32,128,1)
    w, h = img.shape
    flag = False
    
    if h > 128 or w > 32:
        continue
    j += 1
    if w < 32:
        add_zeros = np.ones((32-w, h))*255
        img = np.concatenate((img, add_zeros))
        flag = True
        
    if h < 128:
        add_zeros = np.ones((32, 128-h))*255
        img = np.concatenate((img, add_zeros), axis=1)
        img = np.expand_dims(img , axis = 2)
        flag = True
    
    if h == 128 or w == 32 and not flag:
        img = np.expand_dims(img , axis = 2)
            # Normalize each image
    img = img/255.
    f["test_img"][i, ...] = img[None]
print ('Test data: {}/{}'.format(j,  len(train_img_addr)) )

Train data: 91525/135000
Test data: 10190/135000


In [10]:
for key in f.keys():
    data = f[key]
    print(data[0])

[[[0.21960784]
  [0.19607843]
  [0.18431373]
  ...
  [1.        ]
  [1.        ]
  [1.        ]]

 [[0.20392157]
  [0.19607843]
  [0.20392157]
  ...
  [1.        ]
  [1.        ]
  [1.        ]]

 [[0.19215687]
  [0.2       ]
  [0.22745098]
  ...
  [1.        ]
  [1.        ]
  [1.        ]]

 ...

 [[0.2       ]
  [0.2       ]
  [0.2       ]
  ...
  [1.        ]
  [1.        ]
  [1.        ]]

 [[0.20784314]
  [0.20392157]
  [0.2       ]
  ...
  [1.        ]
  [1.        ]
  [1.        ]]

 [[1.        ]
  [1.        ]
  [1.        ]
  ...
  [1.        ]
  [1.        ]
  [1.        ]]]
Peens
[[[0.972549  ]
  [0.99215686]
  [0.95686275]
  ...
  [1.        ]
  [1.        ]
  [1.        ]]

 [[0.9529412 ]
  [0.9882353 ]
  [0.9882353 ]
  ...
  [1.        ]
  [1.        ]
  [1.        ]]

 [[0.9647059 ]
  [1.        ]
  [0.99215686]
  ...
  [1.        ]
  [1.        ]
  [1.        ]]

 ...

 [[0.9764706 ]
  [0.9764706 ]
  [0.9764706 ]
  ...
  [1.        ]
  [1.        ]
  [1.        ]]

 [

In [11]:
f['train_img'][0].shape

(32, 128, 1)